
(tutorials-hubbard-parallel)=

# Parallelizing the computation of Hubbard parameters

In this tutorial you will learn how to parallelize the computation of the Hubbard parameters using the {py:class}`~aiida_hubbard.workflows.hp.main.HpWorkChain`.

We can divide this goal in two phases:

* __Parallelize over independent atoms__: parallelize the ``hp.x`` calculation with multiple sub-``hp.x`` running single atoms.
* __Parallelize over independent q points__: parallelize each atom sub-``hp.x`` with other sub-``hp.x`` running single q points.

As we learnt from the [previous tutorial](./1_computing_hubbard.ipynb), first we need to compute the ground-state with a ``pw.x`` calculation.

Let's get started!

In [1]:
from local_module import load_temp_profile

# If you download this file, you can run it with your own profile.
# Put these lines instead:
# from aiida import load_profile
# load_profile()
data = load_temp_profile(
    name="hubbard-parallel-tutorial",
    add_computer=True,
    add_pw_code=True,
    add_hp_code=True,
    add_sssp=True,
    add_structure_licoo=True,
)

/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:46: AiidaDeprecationWarning: The `Code` class is deprecated. To create an instance, use the `aiida.orm.nodes.data.code.installed.InstalledCode` or `aiida.orm.nodes.data.code.portable.PortableCode` for a "remote" or "local" code, respectively. If you are using this class to compare type, e.g. in `isinstance`, use `aiida.orm.nodes.data.code.abstract.AbstractCode`. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:58: AiidaDeprecationWarning: The `Code` plugin is deprecated, use the `InstalledCode` (`core.code.remote`) instead. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:493: AiidaDeprecationWarning: `Code.set_remote_computer_exec` method is deprecated, use `InstalledCode`. (this will be removed in v3)
  warn_deprecation('`Code.set_remote_computer_exec` method is 

In [2]:
from aiida.engine import run_get_node
from aiida.orm import KpointsData
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from aiida_quantumespresso.common.types import ElectronicType
kpoints = KpointsData()
kpoints.set_kpoints_mesh([1,1,1])

builder = PwBaseWorkChain.get_builder_from_protocol(
    code=data.pw_code, # modify here if you downloaded the notebook
    structure=data.structure, # modify here if you downloaded the notebook
    protocol="fast",
    electronic_type=ElectronicType.INSULATOR,
    overrides={"kpoints":kpoints, "clean_workdir":False}
)
results, pw_node = run_get_node(builder)

02/18/2025 09:47:42 AM <48797> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [112|PwBaseWorkChain|run_process]: launching PwCalculation<115> iteration #1
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:565: AiidaDeprecationWarning: `Code.is_local` method is deprecated, use a `PortableCode` instance and check the type. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:523: AiidaDeprecationWarning: `Code.get_remote_computer` method is deprecated, use the `computer` attribute instead. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:565: AiidaDeprecationWarning: `Code.is_local` method is deprecated, use a `PortableCode` instance and check the type. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:513: AiidaDeprecationWarn

## Parallelize over atoms

To parallelize over atoms, we need a _new_ workchain which is dedicated to this purpose: the {py:class}`~aiida_hubbard.workflows.hp.main.HpWorkChain`. This workchain is able to parallelize both over atoms and over q points.

Let's see first the atom parallelization. As usual, we need to get the `builder` and fill the inputs.
Specifying the input `parallelize_atoms` as `True` in `HpWorkChain`, each _independent atom_ will be run as a separate `HpBaseWorkChain`.

In [3]:
from aiida_hubbard.workflows.hp.main import HpWorkChain

builder = HpWorkChain.get_builder_from_protocol(
    code=data.hp_code,
    protocol="fast",
    parent_scf_folder=pw_node.outputs.remote_folder,
    overrides={
        "parallelize_atoms":True, 
        "parallelize_qpoints":False, 
        "hp":{"hubbard_structure":data.structure},
        "qpoints_distance": 100.0, # to get few q points
        }
)

results, hp_node = run_get_node(builder)
results

02/18/2025 09:47:56 AM <48797> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [128|HpWorkChain|run_parallel_workchain]: running in parallel, launching HpParallelizeAtomsWorkChain<132>
02/18/2025 09:47:56 AM <48797> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [132|HpParallelizeAtomsWorkChain|run_init]: launched initialization HpBaseWorkChain<134>
02/18/2025 09:47:56 AM <48797> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [134|HpBaseWorkChain|run_process]: launching HpCalculation<137> iteration #1
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:565: AiidaDeprecationWarning: `Code.is_local` method is deprecated, use a `PortableCode` instance and check the type. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:523: AiidaDeprecationWarning: `Code.get_remote_computer` method is deprecated, use the `computer` attribute instead. 

{'hubbard': <Dict: uuid: e97f52a2-a942-43f3-a827-121e3cff18c1 (pk: 177)>,
 'hubbard_chi': <ArrayData: uuid: abae32e5-057c-4cfa-8d33-3cdac0292317 (pk: 179)>,
 'hubbard_matrices': <ArrayData: uuid: 0319f3cc-cd5c-4084-8d9c-7656ddf4f0a9 (pk: 178)>,
 'hubbard_structure': <HubbardStructureData: uuid: 3c1b6c74-1627-4f73-840a-e3866765650b (pk: 180)>,
 'parameters': <Dict: uuid: ab2a083a-bfaa-4e61-b26b-6f6ad796eb80 (pk: 176)>,
 'remote_folder': <RemoteData: uuid: 61aea6f8-cb79-4776-92d3-0d7542e0250b (pk: 174)>,
 'retrieved': <FolderData: uuid: f7079947-c1f1-4c21-b155-06814b2d46f6 (pk: 175)>}

Let's have a look at the workflow:

In [4]:
%verdi process status {hp_node.pk}

HpWorkChain<128> Finished [0] [3:results]
    ├── create_kpoints_from_distance<130> Finished [0]
    └── HpParallelizeAtomsWorkChain<132> Finished [0] [6:results]
        ├── HpBaseWorkChain<134> Finished [0] [3:results]
        │   └── HpCalculation<137> Finished [0]
        ├── HpBaseWorkChain<143> Finished [0] [3:results]
        │   └── HpCalculation<152> Finished [0]
        ├── HpBaseWorkChain<146> Finished [0] [3:results]
        │   └── HpCalculation<155> Finished [0]
        ├── HpBaseWorkChain<149> Finished [0] [3:results]
        │   └── HpCalculation<158> Finished [0]
        └── HpBaseWorkChain<170> Finished [0] [3:results]
            └── HpCalculation<173> Finished [0]


The following just happened:
- A grid of q points is generated automatically using the distance (between points) in $\r{A}^{-1}$ we gave in input (of 100 $\r{A}^{-1}$ to have very sparse - it is just a tutorial!).
- The `HpParallelizeAtomsWorkChain` is called.
- This work chain calls first a `HpBaseWorkChain` to get the independent atoms to perturb.
- **Three** `HpBaseWorkChain` are submitted __simultaneously__, one for cobalt, and two for the two oxygen sites.
- The response matrices ($\chi^{(0)}$,$\chi$) of each atom are collected to post-process them and compute the final U/V values using $V_{IJ} = (\chi^{(0) -1} -\chi^{-1})_{IJ}$

As for the `HpBaseWorkChain`, we also have here the `hubbard_structure` output namespace, containing the same results as the serial execution:

In [5]:
from aiida_quantumespresso.utils.hubbard import HubbardUtils
print(HubbardUtils(results['hubbard_structure']).get_hubbard_card())

HUBBARD	ortho-atomic
 V	Co-3d	Co-3d	1	1	9.8969
 V	Co-3d	O-2p	1	11	3.3429
 V	Co-3d	O-2p	1	22	3.3407



## Parallelize q points for each perturbed atom

In density-functional perturbation theory, we can simulate linear responses in reciprocal space as monocrhomatic perturbations, described via a grid of __q points__: each q point a monocrhomatic perturbation. The number of q points can be reduced using symmetries, and each Hubbard atom (manifold) will have in principle different number of perturbations.

Specifying the input `parallelize_qpoints` as `True` in `HpWorkChain`, each single independent q point _of each atom_ will run as a separate `HpBaseWorkChain`.

:::{important}
To parallelize over q points you __MUST__ parallelize over atoms as well.
:::

In [6]:
builder = HpWorkChain.get_builder_from_protocol(
    code=data.hp_code,
    protocol="fast",
    parent_scf_folder=pw_node.outputs.remote_folder,
    overrides={
        "parallelize_atoms":True, 
        "parallelize_qpoints":True,  
        "hp":{"hubbard_structure":data.structure},
        "qpoints_distance": 1000, # to get few q points
        "max_concurrent_base_workchains": 2, # useful to not overload HPC or local computer
    }
)

results, hp_node = run_get_node(builder)

Report: [189|HpWorkChain|run_parallel_workchain]: running in parallel, launching HpParallelizeAtomsWorkChain<193>
Report: [193|HpParallelizeAtomsWorkChain|run_init]: launched initialization HpBaseWorkChain<195>
Report: [195|HpBaseWorkChain|run_process]: launching HpCalculation<198> iteration #1


/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:565: AiidaDeprecationWarning: `Code.is_local` method is deprecated, use a `PortableCode` instance and check the type. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:523: AiidaDeprecationWarning: `Code.get_remote_computer` method is deprecated, use the `computer` attribute instead. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:565: AiidaDeprecationWarning: `Code.is_local` method is deprecated, use a `PortableCode` instance and check the type. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:513: AiidaDeprecationWarning: `Code.get_remote_exec_path` method is deprecated, use `InstalledCode.filepath_executable` instead. (this will be removed in v3)
  warn_deprecation(


Report: [195|HpBaseWorkChain|results]: work chain completed after 1 iterations
Report: [195|HpBaseWorkChain|on_terminated]: cleaned remote folders of calculations: 198
Report: [195|HpBaseWorkChain|on_terminated]: cleaned remote folders of calculations: 198
Report: [193|HpParallelizeAtomsWorkChain|run_atoms]: launched HpParallelizeQpointsWorkChain<204> for atomic site 1 of kind Co
Report: [193|HpParallelizeAtomsWorkChain|run_atoms]: launched HpParallelizeQpointsWorkChain<207> for atomic site 2 of kind O
Report: [204|HpParallelizeQpointsWorkChain|run_init]: launched initialization HpBaseWorkChain<210>
Report: [207|HpParallelizeQpointsWorkChain|run_init]: launched initialization HpBaseWorkChain<213>
Report: [210|HpBaseWorkChain|run_process]: launching HpCalculation<216> iteration #1
Report: [213|HpBaseWorkChain|run_process]: launching HpCalculation<219> iteration #1


/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:565: AiidaDeprecationWarning: `Code.is_local` method is deprecated, use a `PortableCode` instance and check the type. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:523: AiidaDeprecationWarning: `Code.get_remote_computer` method is deprecated, use the `computer` attribute instead. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:565: AiidaDeprecationWarning: `Code.is_local` method is deprecated, use a `PortableCode` instance and check the type. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:513: AiidaDeprecationWarning: `Code.get_remote_exec_path` method is deprecated, use `InstalledCode.filepath_executable` instead. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3

Report: [210|HpBaseWorkChain|results]: work chain completed after 1 iterations
Report: [210|HpBaseWorkChain|on_terminated]: cleaned remote folders of calculations: 216
Report: [210|HpBaseWorkChain|on_terminated]: cleaned remote folders of calculations: 216
Report: [213|HpBaseWorkChain|results]: work chain completed after 1 iterations
Report: [213|HpBaseWorkChain|on_terminated]: cleaned remote folders of calculations: 219
Report: [213|HpBaseWorkChain|on_terminated]: cleaned remote folders of calculations: 219
Report: [204|HpParallelizeQpointsWorkChain|run_qpoints]: launched HpBaseWorkChain<228> for q point 0
Report: [228|HpBaseWorkChain|run_process]: launching HpCalculation<231> iteration #1
Report: [207|HpParallelizeQpointsWorkChain|run_qpoints]: launched HpBaseWorkChain<234> for q point 0
Report: [234|HpBaseWorkChain|run_process]: launching HpCalculation<237> iteration #1


...

Report: [276|HpBaseWorkChain|results]: work chain completed after 1 iterations
Report: [276|HpBaseWorkChain|on_terminated]: cleaned remote folders of calculations: 279
Report: [276|HpBaseWorkChain|on_terminated]: cleaned remote folders of calculations: 279
Report: [264|HpParallelizeQpointsWorkChain|run_final]: launched HpBaseWorkChain<285> to collect perturbation matrices
Report: [285|HpBaseWorkChain|run_process]: launching HpCalculation<288> iteration #1


/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:565: AiidaDeprecationWarning: `Code.is_local` method is deprecated, use a `PortableCode` instance and check the type. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:523: AiidaDeprecationWarning: `Code.get_remote_computer` method is deprecated, use the `computer` attribute instead. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:565: AiidaDeprecationWarning: `Code.is_local` method is deprecated, use a `PortableCode` instance and check the type. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:513: AiidaDeprecationWarning: `Code.get_remote_exec_path` method is deprecated, use `InstalledCode.filepath_executable` instead. (this will be removed in v3)
  warn_deprecation(


Report: [285|HpBaseWorkChain|results]: work chain completed after 1 iterations
Report: [285|HpBaseWorkChain|on_terminated]: remote folders will not be cleaned
Report: [285|HpBaseWorkChain|on_terminated]: remote folders will not be cleaned
Report: [264|HpParallelizeQpointsWorkChain|on_terminated]: cleaned remote folders of calculations: 270 279 288
Report: [193|HpParallelizeAtomsWorkChain|run_final]: launched HpBaseWorkChain<294> to collect matrices
Report: [294|HpBaseWorkChain|run_process]: launching HpCalculation<297> iteration #1


/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:565: AiidaDeprecationWarning: `Code.is_local` method is deprecated, use a `PortableCode` instance and check the type. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:523: AiidaDeprecationWarning: `Code.get_remote_computer` method is deprecated, use the `computer` attribute instead. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:565: AiidaDeprecationWarning: `Code.is_local` method is deprecated, use a `PortableCode` instance and check the type. (this will be removed in v3)
  warn_deprecation(
/opt/conda/lib/python3.9/site-packages/aiida/orm/nodes/data/code/legacy.py:513: AiidaDeprecationWarning: `Code.get_remote_exec_path` method is deprecated, use `InstalledCode.filepath_executable` instead. (this will be removed in v3)
  warn_deprecation(


Report: [294|HpBaseWorkChain|results]: work chain completed after 1 iterations
Report: [294|HpBaseWorkChain|on_terminated]: remote folders will not be cleaned
Report: [294|HpBaseWorkChain|on_terminated]: remote folders will not be cleaned
Report: [193|HpParallelizeAtomsWorkChain|on_terminated]: cleaned remote folders of calculations: 198 216 231 249 219 237 256 270 279 288 297
Report: [189|HpWorkChain|on_terminated]: cleaned remote folders of calculations: 198 216 231 249 219 237 256 270 279 288 297


In [7]:
%verdi process status {hp_node.pk}

HpWorkChain<189> Finished [0] [3:results]
    ├── create_kpoints_from_distance<191> Finished [0]
    └── HpParallelizeAtomsWorkChain<193> Finished [0] [6:results]
        ├── HpBaseWorkChain<195> Finished [0] [3:results]
        │   └── HpCalculation<198> Finished [0]
        ├── HpParallelizeQpointsWorkChain<204> Finished [0] [5:results]
        │   ├── HpBaseWorkChain<210> Finished [0] [3:results]
        │   │   └── HpCalculation<216> Finished [0]
        │   ├── HpBaseWorkChain<228> Finished [0] [3:results]
        │   │   └── HpCalculation<231> Finished [0]
        │   └── HpBaseWorkChain<246> Finished [0] [3:results]
        │       └── HpCalculation<249> Finished [0]
        ├── HpParallelizeQpointsWorkChain<207> Finished [0] [5:results]
        │   ├── HpBaseWorkChain<213> Finished [0] [3:results]
        │   │   └── HpCalculation<219> Finished [0]
        │   ├── HpBaseWorkChain<234> Finished [0] [3:results]
        │   │   └── HpCalculation<237> Finished [0]
        │   └── H

The following just happened:
- A grid of q points was generated automatically using the distance (between points) in $\r{A}^{-1}$ we gave in input (of 1000 $\r{A}^{-1}$ to have very sparse - it is just a tutorial!).
- The `HpParallelizeAtomsWorkChain` is called.
- This work chain calls first a `HpBaseWorkChain` to get the independent atoms to perturb.
- For independent each atom (three in total) an `HpParallelizeQpointsWorkChain` is submitted __simultaneously__, one for cobalt, and two for the two oxygen sites.
- Each of such work chain submit a fist `HpBaseWorkChain` to get the independent q points (in this case, only 1).
- An `HpBaseWorkCahin` is run for every q points, executed at the same time. __Imagine this on an HPC!__ :rocket:
- The response matrices ($\chi^{(0)}_{\mathbf{q}}$,$\chi_{\mathbf{q}}$) of each q point for each atom are collected to post-process them and compute the atomic response matrices.
- A last final `HpBaseWorkChain` collects such matrices to compute U/V values.

And we check the results are the same as before:

In [8]:
print(HubbardUtils(results['hubbard_structure']).get_hubbard_card())

HUBBARD	ortho-atomic
 V	Co-3d	Co-3d	1	1	9.8969
 V	Co-3d	O-2p	1	11	3.3429
 V	Co-3d	O-2p	1	22	3.3407



## Final considerations

We managed to compute the Hubbard parameters __parallelizing over atoms and q points__! :tada:

Still, you might need to converge self-consistently the parameters using the iterative procedure of relax -> scf -> hubbard.
Learn the automated way [in the last tutorial](./3_self_consistent.ipynb)!

:::{admonition} Learn more and in details
:class: hint

To learn the full sets of inputs, to use proficiently the `get_builder_from_protocol` and more, have a look at the following sections:
- [Specific how tos](howto/workflows/hp/main.md)
- [General information of the implemented workchain](topics/workflows/hp/main.md)
:::